### Тестовое задание на 20.11
#### Выполнил студент группы 9381 Матвеев А. 

Цели:
1. Индивидуальное творческое соревнование.
2. Самостоятельное освоение одного из важных методов обучения без учителя — обучение без учителя.
3. Практическое применение дискретного преобразования Фурье для извлечения признаков обучения.
4. Использованием библиотек Python для кластеризации музыкальных сигналов.
    
Выполнение:

Алгоритм К-means:
1. Задаем количество кластеров, k (положим k = 5).
2. Инициализация центроидов. Случайным образом выбираются k точек данных и объявляются центрами кластеров (центроидами). Здесь возможны различные стратегии выбора (творческий выбор).
3. Разбиение на кластеры. Для каждого элемента набора определяется ближайший к нему центр кластера (расстояния измеряются в метрике Евклида).
4. Вычисление новых центроидов Новые центроиды — это вектора, компоненты которых представляют собой средние значения соответствующих признаков, вычисленные по всем элементам соответствующего кластера.
5. Повторяем шаги 3 — 4



In [34]:

import numpy as np
import scipy
import more_itertools as mit
import sklearn.cluster
import csv
import scipy.signal as spectrogram
import IPython.display as ipd
import matplotlib.pyplot as plt
import random
import copy
from pydub import AudioSegment
import random
import copy

%matplotlib inline



In [35]:
trackName = "./mp_3/0{}.mp3"
trackName1 = "mp_3/{}.mp3"
tracks = []
signals = []
titles=[]

for i in range(1, 125): # считываем
    if(i < 10):
        #print(trackName.format(i))
        sound = AudioSegment.from_mp3(trackName.format(i));
        titles.append(trackName.format(i+1))
    else:
        sound = AudioSegment.from_mp3(trackName1.format(i));
        titles.append(trackName1.format(i+1))
    tracks.append(sound)
    signal = sound.get_array_of_samples()
    signals.append(signal)
    

bitrate = 256 # битрейт
offset = 256   

In [36]:
spec_ress = [] # спектральные разрешения
for i in range(len(tracks)):
    spec_ress.append(tracks[i].frame_rate // bitrate)
#print(spec_ress)

In [42]:
amplit_specters = []

for signal in signals:
    inters = list(mit.windowed(signal, fillvalue=0, n=bitrate, step=offset))
    max_ = 0 
    for i in inters:
        m = np.max(i)
        if m > max_:
            max_ = m
    max_ *= 0.2

    inters = list(filter(lambda i: np.max(i) > max_, inters))
    #print(len(inters))
    inters = list(map(lambda intl: scipy.fft.fft(intl * np.hamming(len(intl))), inters))
    ampl = [[np.abs(i) for i in intl[:bitrate//2]] for intl in inters]


    a = [0 for i in range(bitrate // 2)]
    for i in ampl:
        for inner in range(bitrate // 2):
            a[inner] += i[inner]

    for i in range(bitrate // 2):
        a[i] /= len(inters)
    
    amplit_specters.append(a)



                       

In [45]:


tab_ind = [0 for i in range(len(amplit_specters))]



def count_min_dist(data, cnts, tab_ind): # функция которая считает для каждой точки минимальное расстояние до центроида других кластеров
    bitrate = len(data[0])
    n = len(data)
    k = len(cnts)
    clusterArray = [[] for i in range(k)]

    for i in range(n):
        mindist = float('inf')
        approved_cl = -1
        
        for j in range(k):
            dist = 0
            for f in range(bitrate):
                dist += (data[i][f] - cnts[j][f]) ** 2
            dist = dist ** 0.5
            
            if dist < mindist:
                mindist = dist
                approved_cl = j
            
        tab_ind[i] = approved_cl;
        clusterArray[approved_cl].append(i)
    return clusterArray

def recountCentroids(data, clusterArray, cnts): # пересчёт центроидов берём среднее из точек которые принадлежат
    bitrate = len(data[0])
    k = len(cnts)
    
    for i in range(k):
        for f in range(bitrate): 
            if len(clusterArray[i]) != 0:
                updt = 0
                for j in range(len(clusterArray[i])): 
                    updt += data[clusterArray[i][j]][f]
                updt = updt / len(clusterArray[i])
                cnts[i][f] = updt
                
def clusterization(data, k, tab_ind):
    n = len(data)  

    inds = []
    for i in range(k):
        r = random.randint(0, k - 1)
        while r in inds:
            r = random.randint(0, k - 1)
        inds.append(r)
        
    centroids = [copy.deepcopy(data[inds[i]]) for i in range(k)]
    clusterArray = count_min_dist(data, centroids, tab_ind)
    prevs = None
        
    while True:
        if centroids == prevs:
            break;
        prevs = copy.deepcopy(centroids)
        recountCentroids(data, clusterArray, centroids)
        clusterArray = count_min_dist(data, centroids, tab_ind)
    
    
    k = len(clusterArray) 
    CC = 0
    for i in range(k):
        for j in range(len(clusterArray[i])):
            CC+= abs(clusterArray[i][j] - np.average(centroids[i])) ** 2
    
    CC /= k # компакность центроидов
    
    return (clusterArray, CC)    


print('Искомая кластеризация: ')
res = clusterization(amplit_specters, 5, tab_ind);
print(res[0])
print('Компактность кластеров: ', round(res[1], 3))



Искомая кластеризация: 
[[24, 38, 39, 43, 44, 45, 46, 47, 52, 55, 63, 64, 66, 69, 71, 73, 74, 75, 76, 77, 78, 81, 83, 93, 95, 109, 123], [13, 15, 20, 21, 22, 35, 36, 37, 42, 51, 98, 102, 105, 107, 108], [1, 7, 10, 27, 41, 53, 54, 56, 57, 58, 60, 61, 62, 79, 82, 85, 86, 88, 90, 92, 94, 103], [12, 17, 18, 19, 23, 25, 26, 29, 31, 32, 33, 40, 48, 49, 50, 59, 68, 70, 72, 80, 84, 87, 89, 91, 96, 101, 104, 111, 121], [0, 2, 3, 4, 5, 6, 8, 9, 11, 14, 16, 28, 30, 34, 65, 67, 97, 99, 100, 106, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122]]
Компактность кластеров:  2740115925.829


In [44]:
with open('new.csv', 'w', newline='') as  file:
    pen = csv.writer(file, delimiter=',')
    for i in range(len(tab_ind)):
        pen.writerow([titles[i], tab_ind[i]])

### Выводы:
    Был написан алгоритм, позволяющий разбить аудиотреки на кластеры. Была построена csv-таблица с результатами, изучен алгоритм K-means, построена метрика качества.